<a href="https://colab.research.google.com/github/YudaiFukushige/PINN_Hydraulics/blob/main/PINN_1DOpenChannelFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# 問題設定

- 水路の形状
    - 勾配 : 1/1000
    - 横断面 : 矩形
    - 水路幅 : 1 m
    - 粗度係数 : 0.015
-





# Loss functions

In [11]:
criteria = nn.MSELoss()

## Physics informed loss

In [14]:
def physics_informed_loss_mass(x, t, net):
    """
    連続式に関するPhysics informed lossを計算する関数
    x : 空間
    t : 時間
    net : (x, t)を受け取って，(h, u)を返すネットワーク
    """
    # 予測値を計算
    h = net(x, t)[:, 0]
    u = net(x, t)[:, 1]
    # 水深 h の x による1階偏微分
    h_x = torch.autograd.grad(
        h, x,
        grad_outputs=torch.ones_like(h),
        retain_graph=True,
        create_graph=True,
        allow_unused=True
    )[0]
    # 水深 h の t による1階偏微分
    h_t = torch.autograd.grad(
        h, t,
        grad_outputs=torch.ones_like(h),
        retain_graph=True,
        create_graph=True,
        allow_unused=True
    )[0]
    # 流速 u の x による1階偏微分
    u_x = torch.autograd.grad(
        u, x,
        grad_outputs=torch.ones_like(u),
        retain_graph=True,
        create_graph=True,
        allow_unused=True
    )[0]
    # Physics informed loss
    pinn_loss_mass = h_t + u*h_x + h*u_x
    zeros_t = torch.zeros(pinn_loss_mass.size()).to(device)
    pinn_loss_mass_ = criteria(pinn_loss_mass, zeros_t)
    return pinn_loss_mass_

In [1]:
def physics_informed_loss_momentum(x, t, i0, B, n, net):
    """
    運動方程式に関するPhysics informed lossを計算する関数
    x : 空間
    t : 時間
    i0 : 勾配
    B : 水路幅
    n : マニングの粗度係数
    net : (x, t)を受け取って，(h, u)を返すネットワーク
    """
    # 予測値を計算
    h = net(x, t)[:, 0]
    u = net(x, t)[:, 1]
    # 水深 h の x による1階偏微分
    h_x = torch.autograd.grad(
        h, x,
        grad_outputs=torch.ones_like(h),
        retain_graph=True,
        create_graph=True,
        allow_unused=True
    )[0]
    # 流速 u の x による1階偏微分
    u_x = torch.autograd.grad(
        u, x,
        grad_outputs=torch.ones_like(u),
        retain_graph=True,
        create_graph=True,
        allow_unused=True
    )[0]
    # 流速 u の t による1階偏微分
    u_t = torch.autograd.grad(
        u, t,
        grad_outputs=torch.ones_like(u),
        retain_graph=True,
        create_graph=True,
        allow_unused=True
    )[0]
    # 径深
    R = culclate_hydraulic_radius(h=h, B=B)
    # 摩擦損失高
    friction_term = (n**2 * u**2) / R**(4/3)

    # Physics informed loss
    pinn_loss_momentum = -i0 + h_x + (1/9.8)*u_x + friction_term + (1/9.8)*u_t
    zeros_t = torch.zeros(pinn_loss_momentum.size()).to(device)
    pinn_loss_momentum_ = criteria(pinn_loss_momentum, zeros_t)
    return pinn_loss_momentum_

In [9]:
def culclate_hydraulic_radius(h, B=1.0):
    """
    一様な矩形水路の径深を計算する
    """
    return B + 2.0*h

## Initial condition

## Boundary condition